In [ ]:
import random
import math
from copy import deepcopy

# Component definitions (name, width, height)
COMPONENTS = [
    ("ALU", 5, 5),
    ("Cache", 7, 4),
    ("Control Unit", 4, 4),
    ("Register File", 6, 6),
    ("Decoder", 5, 3),
    ("Floating Unit", 5, 5)
]

COMPONENT_INDICES = {
    "ALU": 0,
    "Cache": 1,
    "Control Unit": 2,
    "Register File": 3,
    "Decoder": 4,
    "Floating Unit": 5
}

# Grid size
GRID_SIZE = 25

# Wiring pairs (by index)
WIRING_PAIRS = [
    (COMPONENT_INDICES["Register File"], COMPONENT_INDICES["ALU"]),
    (COMPONENT_INDICES["Control Unit"], COMPONENT_INDICES["ALU"]),
    (COMPONENT_INDICES["ALU"], COMPONENT_INDICES["Cache"]),
    (COMPONENT_INDICES["Register File"], COMPONENT_INDICES["Floating Unit"]),
    (COMPONENT_INDICES["Cache"], COMPONENT_INDICES["Decoder"]),
    (COMPONENT_INDICES["Decoder"], COMPONENT_INDICES["Floating Unit"])
]

# Fitness weights
ALPHA = 1000  # Overlap penalty
BETA = 2      # Wiring penalty
GAMMA = 1     # Bounding area penalty

In [ ]:
def random_chromosome():
    """Generate a random chromosome (layout)."""
    chrom = []
    for _, w, h in COMPONENTS:
        x = random.randint(0, GRID_SIZE - w)
        y = random.randint(0, GRID_SIZE - h)
        chrom.append((x, y))
    return chrom

def overlap_count(chrom):
    """Count overlapping component pairs."""
    count = 0
    for i in range(len(COMPONENTS)):
        x1, y1 = chrom[i]
        w1, h1 = COMPONENTS[i][1], COMPONENTS[i][2]
        for j in range(i+1, len(COMPONENTS)):
            x2, y2 = chrom[j]
            w2, h2 = COMPONENTS[j][1], COMPONENTS[j][2]
            if not (x1 + w1 <= x2 or x2 + w2 <= x1 or y1 + h1 <= y2 or y2 + h2 <= y1):
                count += 1
    return count

def wiring_length(chrom):
    """Calculate total wiring length between specified pairs."""
    def center(idx):
        x, y = chrom[idx]
        w, h = COMPONENTS[idx][1], COMPONENTS[idx][2]
        return (x + w/2, y + h/2)
    total = 0.0
    for i, j in WIRING_PAIRS:
        cx1, cy1 = center(i)
        cx2, cy2 = center(j)
        total += math.hypot(cx1 - cx2, cy1 - cy2)
    return total

def bounding_box_area(chrom):
    """Calculate area of the bounding box containing all components."""
    xs, ys = [], []
    for idx, (x, y) in enumerate(chrom):
        w, h = COMPONENTS[idx][1], COMPONENTS[idx][2]
        xs.extend([x, x + w])
        ys.extend([y, y + h])
    return (max(xs) - min(xs)) * (max(ys) - min(ys))

def fitness(chrom):
    """Compute fitness value (lower is better)."""
    overlaps = overlap_count(chrom)
    wire = wiring_length(chrom)
    area = bounding_box_area(chrom)
    return - (ALPHA * overlaps + BETA * wire + GAMMA * area)

In [ ]:
def mutate(chrom, mutation_rate=0.08):
    """Randomly mutate one component's position."""
    if random.random() < mutation_rate:
        idx = random.randint(0, len(COMPONENTS) - 1)
        w, h = COMPONENTS[idx][1], COMPONENTS[idx][2]
        x = random.randint(0, GRID_SIZE - w)
        y = random.randint(0, GRID_SIZE - h)
        chrom[idx] = (x, y)
    return chrom

def select_parents(population):
    """Randomly select two parents."""
    return random.sample(population, 2)

In [ ]:
def single_point_crossover(parent1, parent2):
    """Single-point crossover between two parent chromosomes."""
    point = random.randint(1, len(COMPONENTS) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def genetic_algorithm_single_point(generations=15, pop_size=6):
    # Initialize population
    population = []
    for i in range(pop_size):
       chromosome = random_chromosome()
       population.append(chromosome)
    # population = [random_chromosome() for _ in range(pop_size)]
    for gen in range(generations):
        # Evaluate fitness
        population.sort(key=fitness, reverse=True)
        next_gen = population[:2]  # Elitism: carry top 2
        while len(next_gen) < pop_size:
            p1, p2 = select_parents(population)
            c1, c2 = single_point_crossover(deepcopy(p1), deepcopy(p2))
            mutate(c1)
            mutate(c2)
            next_gen.extend([c1, c2])
        population = next_gen[:pop_size]
    # Final best solution
    best = max(population, key=fitness)
    return {
        "fitness": fitness(best),
        "overlaps": overlap_count(best),
        "wiring_length": wiring_length(best),
        "bounding_area": bounding_box_area(best),
        "coordinates": best
    }

In [ ]:
def two_point_crossover(parent1, parent2):
    """Two-point crossover between two parent chromosomes."""
    p1, p2 = sorted(random.sample(range(1, len(COMPONENTS)), 2))
    child1 = parent1[:p1] + parent2[p1:p2] + parent1[p2:]
    child2 = parent2[:p1] + parent1[p1:p2] + parent2[p2:]
    return child1, child2

def genetic_algorithm_two_point(generations=15, pop_size=6):
    population = []
    for i in range(pop_size):
       chromosome = random_chromosome()
       population.append(chromosome)
    # population = [random_chromosome() for _ in range(pop_size)]
    for gen in range(generations):
        population.sort(key=fitness, reverse=True)
        next_gen = population[:2]
        while len(next_gen) < pop_size:
            p1, p2 = select_parents(population)
            c1, c2 = two_point_crossover(deepcopy(p1), deepcopy(p2))
            mutate(c1)
            mutate(c2)
            next_gen.extend([c1, c2])
        population = next_gen[:pop_size]
    best = max(population, key=fitness)
    return {
        "fitness": fitness(best),
        "overlaps": overlap_count(best),
        "wiring_length": wiring_length(best),
        "bounding_area": bounding_box_area(best),
        "coordinates": best
    }

In [ ]:
# Task 01: Single-point crossover
result1 = genetic_algorithm_single_point()
print("Task 01 (Single-point):", result1)

# Task 02: Two-point crossover
result2 = genetic_algorithm_two_point()
print("Task 02 (Two-point):", result2)

Task 01 (Single-point): {'fitness': -658.3087515207743, 'overlaps': 0, 'wiring_length': 76.15437576038715, 'bounding_area': 506, 'coordinates': [(5, 8), (9, 0), (21, 7), (2, 2), (16, 19), (16, 7)]}
Task 02 (Two-point): {'fitness': -511.171151118678, 'overlaps': 0, 'wiring_length': 60.085575559339, 'bounding_area': 391, 'coordinates': [(16, 19), (18, 4), (14, 13), (8, 18), (11, 6), (11, 1)]}


In [ ]:
a,b = 26,13
c = bin(26)[2:]
d = bin(13)
print(c)

11010


In [ ]:
#bit battle
def  bit_battle(player_1,player_2):
     player_1_bin = format(player_1, '010b')[::-1]
     player_2_bin = format(player_2, '010b')
     p1,p2,draw = 0,0,0
     for i in range(10):
       if player_1_bin[i] == player_2_bin[i]:
           draw  += 1
       elif player_1_bin[i] == '1' and player_2_bin[i] == '0':
           p1 += 1
       else:
           p2 += 1
     return p1,p2,draw
Ruhan =899
arman = 450
print(bit_battle(Ruhan,arman))


(3, 2, 5)


In [ ]:
string = str(input())
vowel = "AEIOUaeiou"
count = 0
for i in string:
  if i in vowel:
    count += 1
print(count)

fdsijjfbalgfbeqirgbevnjsadm vjsavpiyeqfgb[eqf savc nxsavcpsbauvf[dvfbsdvsajkvc
15
